In [ ]:
!pip install dulwich --global-option="--pure"

In [ ]:
import os
import uuid
import json
import docker
from dulwich.repo import Repo


In [ ]:
# set up Docker client and remote Docker Machine connection
tls_config = docker.tls.TLSConfig(
    client_cert=('wrnaz/cert.pem', 'wrnaz/key.pem'),
    verify='wrnaz/ca.pem'
)
client = docker.DockerClient(base_url='tcp://23.97.131.169:2376', tls=tls_config)


In [ ]:
sessions_path = '/home/jovyan/warren/faas/faas_sessions/'
notebook = ''

def create_session(session_type):
    # create a folder with a unique name
    if session_type == 'nb':
        id = str(uuid.uuid4())
        if not os.path.exists(sessions_path+id):
            os.makedirs(sessions_path+id)
            
            # make the folder a bit more useful with `git init`
            print(sessions_path+id)
            session_repo = Repo.init(sessions_path+id)
            
            return id
        else:
            return Exception(FileExistsError)
        
    


In [ ]:
#handle Jupyter notebook executions
#BUT DO IT FUCKING LATER WHEN THERE'S TIME!
def store_notebook(path, fileName, data):
    filePathNameWExt = path + '/' + fileName
    print(data)
    with open(filePathNameWExt, 'w+') as fp:
        fp.write(data)
    fp.close()

def run_notebook(session_id):
    container = client.containers.run('jupyter/scipy-notebook', 'jupyter notebook '+ sessions_path + session_id + '/nb.ipynb')
    return container.id

In [ ]:
# handle Python script execution
def store_script(path, fileName, data):
    filePathNameWExt = path + '/' + fileName
    print(filePathNameWExt)
    with open(filePathNameWExt, 'w+') as fp:
        fp.write(data)
    fp.close()

def run_script(session_id):
    container = client.containers.run('jupyter/scipy-notebook', 'jupyter notebook '+ sessions_path + session_id + '/nb.ipynb')
    return container.id

In [ ]:
# prepare the JSON schema (or metadata) so we could make it as easy as possible
# but also not forget anything important our users would miss. 

# REMEMBER! this is currently only to send our notebooks, 
# scripts, apps, to the cloud for execution
REQUEST = json.dumps({
    'body' : {}
})

In [ ]:
# POST /notebook
try:
    req = json.loads(REQUEST)
    notebook = req['body']
    session_id = create_session('nb')
    store_notebook(sessions_path+session_id, 'nb.ipynb', notebook)
    response = run_notebook(session_id)
except Exception as e:
    response = e
finally:
    print(response)
